In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('online_shoppers_intention.csv')
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [3]:
data.shape

(12330, 18)

In [4]:
# Checking for NaN Values
data.isna().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [5]:
col_l = []
for col in data.columns:
    col = col.replace(' ','_')
    col_l.append(col.lower())
data.columns = col_l
print(data.columns)

Index(['administrative', 'administrative_duration', 'informational',
       'informational_duration', 'productrelated', 'productrelated_duration',
       'bouncerates', 'exitrates', 'pagevalues', 'specialday', 'month',
       'operatingsystems', 'browser', 'region', 'traffictype', 'visitortype',
       'weekend', 'revenue'],
      dtype='object')


In [6]:
data['month'].value_counts()

May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: month, dtype: int64

In [7]:
data['revenue'].value_counts()

False    10422
True      1908
Name: revenue, dtype: int64

In [8]:
# Make a copy of data for use in Tableau visualization
# The original data set shall be used in finding corellations and possibly building a model
# Assign numbers in 'operatingsystems', 'browser' and 'region' actual values
data1 = data.copy()

In [ ]:
# Mapping operating systems integers to values
# ASSUMPTIONS
# 1. There are 8 different operting systems with a popularity trend similar to computers. Phones and tablets have a different 
# popularity trend depending on the country
# 2. The trends are not an exact match but will map them using their popularity rankings
# 3. Could the various browsers being used indicate computer usage rather than phone/tablet?? Usually phone/tablet users would likely
# downlaod the app. Every user uses atleast a browser

# Link: https://en.wikipedia.org/wiki/Usage_share_of_operating_systems

In [9]:
data1['operatingsystems'] = data1['operatingsystems'].map({2:'Windows',1:'Mac OS', 3:'Linux',4:'Ubuntu', 5:'others',6:'others',7:'others',8:'Unix'})
data1['operatingsystems'].value_counts()

Windows    6601
Mac OS     2585
Linux      2555
Ubuntu      478
Unix         79
others       32
Name: operatingsystems, dtype: int64

In [10]:
# LINK TO BROWSER USAGE - Case of the European market: https://backlinko.com/browser-market-share
def clean_browser(x):
    if x == 2:
        return 'Chrome'
    elif x == 1:
        return 'Safari'
    elif x == 4:
        return 'Mozilla'
    elif x == 5:
        return 'Opera'
    elif x ==6:
        return 'Edge'
    elif x == 10:
        return 'Yandex'
    elif x == 8:
        return 'WebView'
    elif x == 3:
        return 'Samsung'
    else:
        return 'Others'

data1['browser'] = data1['browser'].apply(lambda x:clean_browser(x))
data1['browser'].value_counts()

Chrome     7961
Safari     2462
Mozilla     736
Opera       467
Edge        174
Yandex      163
WebView     135
Others      127
Samsung     105
Name: browser, dtype: int64

In [11]:
# Regions are totally abitrary because i do not know where the store is located
data1['region'] = data1['region'].map({1:'Central',2:'Northern', 3:'Eastern', 4:'Western',7:'Southern',6:'North-West',9:'North-East',
                                      8:'South-West',5:'South-East'})
data1['region'].value_counts()

Central       4780
Eastern       2403
Western       1182
Northern      1136
North-West     805
Southern       761
North-East     511
South-West     434
South-East     318
Name: region, dtype: int64

In [12]:
# LINK: https://www.cyberclick.net/numericalblogen/types-of-web-traffic-sources-and-explanations
# Got the types of traffic from the above link. I did not consider the percentages because i do not know how our
# store generated most of its traffic

def clean_traffic(x):
    if x == 2:
        return 'Organic'
    elif x == 1:
        return 'Direct'
    elif x == 3:
        return 'Referral'
    elif x == 4:
        return 'Email marketing'
    elif x == 13:
        return 'Social networks'
    elif x == 10:
        return 'Paid media'
    elif x == 6:
        return 'Paid Search'
    elif x == 8:
        return 'Offline channels'    
    else:
        return 'Others'

data1['traffictype'] = data1['traffictype'].apply(lambda x:clean_traffic(x))
data1['traffictype'].value_counts()

Organic             3913
Direct              2451
Referral            2052
Email marketing     1069
Others               870
Social networks      738
Paid media           450
Paid Search          444
Offline channels     343
Name: traffictype, dtype: int64

In [13]:
# People taht visited the website and did not buy anything are classified as visitors
visitors = data1[data1['revenue'] == False]
visitors.shape

(10422, 18)

In [14]:
# People taht visited the website and bought item(s) are classified as customers
customers = data1[data1['revenue'] == True]
customers.shape

(1908, 18)

In [15]:
# PERCENTAGE OF RETURNING VISITORS THAT CONVERTED - SPENT MONEY
percent_buy = len(customers)/len(data1)
print("Ratio of visitors that bought items ", round(percent_buy,2))

Ratio of visitors that bought items  0.15


In [16]:
# Use data where 'Revenue' is True
customers['visitortype'].value_counts()

Returning_Visitor    1470
New_Visitor           422
Other                  16
Name: visitortype, dtype: int64

In [48]:
data1.to_csv("E:\\SKUL\\PROGRAMMING_SKULS\\ironhack\\COURSE\\Mid-Term-Project\\Data\\Visitors.csv", index = False)

In [17]:
# Export dataframe where 'revenue' is True for use in Tableau
customers.to_excel("E:\\SKUL\\PROGRAMMING_SKULS\\ironhack\\COURSE\\Mid-Term-Project\\Data\\Customers.xlsx")

In [18]:
# Mothly Breakdown of customers that bought items 
customers['month'].value_counts()

Nov     760
May     365
Dec     216
Mar     192
Oct     115
Sep      86
Aug      76
Jul      66
June     29
Feb       3
Name: month, dtype: int64

In [19]:
# Percentage of revenue from returning visitors
perc_return_customer_revenue = len(customers[customers['visitortype'] == 'Returning_Visitor'])/len(customers)
print("Ratio of returning customers that buy items: ", round(perc_return_customer_revenue, 2))

Ratio of returning customers that buy items:  0.77


In [20]:
# percentage of revenue from new visitors
perc_revenue_new_customer = len(customers[customers['visitortype'] == 'New_Visitor'])/len(customers)

print("Ratio of new customers that buy items: ", round(perc_revenue_new_customer, 2))

Ratio of new customers that buy items:  0.22


In [21]:
# Percentage of returning visitors that didnot buy
# We use the dataframe where 'revenue' is False
no_revenue_return_customer = len(visitors[visitors['visitortype'] == 'Returning_Visitor'])/len(visitors)
round(no_revenue_return_customer, 2)

0.87

In [22]:
# This percentage could be that returning customers simply make up a huge chunk of our website visitors
overall_return = len(data[data['visitortype'] == 'Returning_Visitor'])/len(data1)
print("Returning Customers Percentage: ", round(overall_return,2))

Returning Customers Percentage:  0.86


In [23]:
# from the above calculation, 85.5% of visitors are returning customers

In [24]:
# New customers
overall_new = len(data1[data1['visitortype'] == 'New_Visitor'])/len(data1)
print("New Customers Percentage: ", round(overall_new,2))

New Customers Percentage:  0.14


In [25]:
# Revenue on special days
special_days = data1[data1['specialday'] == 1]
special_days.head()

,administrative,administrative_duration,informational,informational_duration,productrelated,productrelated_duration,bouncerates,exitrates,pagevalues,specialday,month,operatingsystems,browser,region,traffictype,visitortype,weekend,revenue
20,0,0.0,0,0.0,8,136.166667,0.0,0.008333,0.000000,1.0,Feb,Windows,Chrome,South-East,Direct,Returning_Visitor,True,False
52,0,0.0,0,0.0,2,29.000000,0.0,0.100000,0.000000,1.0,Feb,Windows,Mozilla,Western,Organic,Returning_Visitor,True,False
78,0,0.0,0,0.0,1,0.000000,0.2,0.200000,0.000000,1.0,Feb,Mac OS,Safari,Central,Referral,Returning_Visitor,True,False
101,4,61.0,0,0.0,19,607.000000,0.0,0.026984,17.535959,1.0,Feb,Mac OS,Safari,Southern,Email marketing,Returning_Visitor,True,True
174,5,41.3,0,0.0,24,446.927778,0.0,0.008602,0.000000,1.0,Feb,Windows,Chrome,South-West,Direct,Returning_Visitor,True,False


In [26]:
percentage_of_special_day = len(special_days)/len(data1)
print("Ratio of visitors on special days: ",round(percentage_of_special_day,2) )

Ratio of visitors on special days:  0.01


In [27]:
special_revenue_perc = len(special_days[special_days['revenue'] == True])/len(special_days)
print("Ratio of revenue customers on special days: ",round(special_revenue_perc,2))

Ratio of revenue customers on special days:  0.06


In [28]:
# Only 6% of visitors on special day converted

In [29]:
data1['weekend'].value_counts()

False    9462
True     2868
Name: weekend, dtype: int64

In [30]:
# Visitors over the weekend
weekend_visitors = data1[data1['weekend'] == True]
weekend_visitors.shape

(2868, 18)

In [31]:
percentage_of_weekend_visitors = len(weekend_visitors)/len(data1)
print("Ratio of weekend visitors: ", round(percentage_of_weekend_visitors,2))

Ratio of weekend visitors:  0.23


In [32]:
# Checking conversion ratios for both weekend and weekdays
# Getting weekend visitors that bought items
weekend_customers = weekend_visitors[weekend_visitors['revenue'] == True]
ratio_of_weekend_customers = len(weekend_customers)/len(weekend_visitors)
print("Ratio of weekend customers: ", round(ratio_of_weekend_customers,2))

Ratio of weekend customers:  0.17


In [33]:
# Ratios are similar between overall customer conversion rates and weekend customer conversion rates

In [34]:
# Visitors in weekdays
weekdays_visitors = data1[data1['weekend'] == False]
weekdays_visitors.shape

(9462, 18)

In [35]:
weekdays_customers = weekdays_visitors[weekdays_visitors['revenue'] == True]
weekdays_customers.shape

(1409, 18)

In [36]:
ratio_of_weekdays_customers = len(weekdays_customers)/len(weekdays_visitors)
print("Ratio of weekdays customers: ", round(ratio_of_weekdays_customers,3))

Ratio of weekdays customers:  0.149


In [37]:
# How many customers do we get by a 1% increase in visitors
extra_customers = 0.01 * 12330 * 0.15
print("Extra customers from a 1% increase in traffic: ", int(extra_customers))

Extra customers from a 1% increase in traffic:  18


In [38]:
# Because of data limitations, i can not tell whether the increase in customers is significant
# Since this is e-commerce, i will assume it is not so significant

In [39]:
# Check regions with highest customers
customers['region'].value_counts()

Central       771
Eastern       349
Northern      188
Western       175
Southern      119
North-West    112
North-East     86
South-West     56
South-East     52
Name: region, dtype: int64

In [70]:
data['operatingsystems'].value_counts()

2    6601
1    2585
3    2555
4     478
8      79
6      19
7       7
5       6
Name: operatingsystems, dtype: int64

In [69]:
data1['operatingsystems'].value_counts()

Windows    6601
Mac OS     2585
Linux      2555
Ubuntu      478
Unix         79
others       32
Name: operatingsystems, dtype: int64

In [68]:
my_data.groupby(['operatingsystems','browser']).size()

operatingsystems  browser
Linux             Chrome     2416
                  Edge         11
                  Opera         6
                  Others       17
                  Safari        1
                  Samsung     104
Mac OS            Chrome      280
                  Edge          4
                  Mozilla       3
                  Opera         5
                  Others        2
                  Safari     2153
                  WebView     135
                  Yandex        3
Ubuntu            Chrome      174
                  Edge          3
                  Mozilla       1
                  Opera        21
                  Safari      279
Unix              Chrome       17
                  Edge          1
                  Opera         2
                  Others       53
                  Safari        6
Windows           Chrome     5059
                  Edge        155
                  Mozilla     732
                  Opera       430
                  Othe